.

Data_Engineering_TIL(20201206)

study program : T아카데미 - 아파치 스파크 입문과 활용

** URL : https://tacademy.skplanet.com/frontMain.action

### [학습내용]

step 1) 실습환경 구성


spark을 띄우기 위해서 spark cluster를 구성해야하는데 시간상 제플린 환경으로 실습한다. 제플린에서 spark context를 띄워서 실습해볼 수 있다.

참고로 spark을 설치한다는 것은 spark만 설치하면 끝나는게 아니라 yarn과 Hadoop을 설치를 해주고 연동해줘야 한다.


아래와 같은 spec으로 aws ec2 생성

In [ ]:
aws ec2 run-instances --image-id ami-03b42693dc6a7dc35 --count 1 --instance-type t3.medium --key-name pms-seoul-key --security-group-ids sg-xxxxxxxxxxxx --subnet-id subnet-xxxxxxxxxxxxx --associate-public-ip-address --tag-specifications 'ResourceType=instance,Tags=[{Key=Name,Value=pms-spark-test},{Key=owner,Value=pms},{Key=expiry-date,Value=2020-12-04}]' --block-device-mappings 'DeviceName=/dev/xvda,Ebs={VolumeSize=30,DeleteOnTermination=true}'

그런 다음에 해당 ec2로 ssh 접속해서 아래와 같은 명령어 실행

In [ ]:
[ec2-user@ip-10-0-1-170 ~]$ sudo yum update -y

[ec2-user@ip-10-0-1-170 ~]$ sudo amazon-linux-extras install docker -y

[ec2-user@ip-10-0-1-170 ~]$ sudo service docker start
Redirecting to /bin/systemctl start docker.service

[ec2-user@ip-10-0-1-170 ~]$ sudo usermod -a -G docker ec2-user

[ec2-user@ip-10-0-1-170 ~]$ exit

터미널 로그아웃 후 다시 ec2로 재접속하고 아래와 같이 명령어를 실행했을때 sudo 명령어를 안줘도 되는지 확인해본다.

In [ ]:
[ec2-user@ip-10-0-1-170 ~]$ docker info
Client:
 Debug Mode: false

Server:
 Containers: 0
  Running: 0
  Paused: 0
  Stopped: 0
 Images: 0
 Server Version: 19.03.13-ce
 Storage Driver: overlay2
  Backing Filesystem: xfs
  Supports d_type: true
  Native Overlay Diff: true
 Logging Driver: json-file
 Cgroup Driver: cgroupfs
 Plugins:
  Volume: local
  Network: bridge host ipvlan macvlan null overlay
  Log: awslogs fluentd gcplogs gelf journald json-file local logentries splunk syslog
 Swarm: inactive
 Runtimes: runc
 Default Runtime: runc
 Init Binary: docker-init
 containerd version: c623d1b36f09f8ef6536a057bd658b3aa8632828
 runc version: ff819c7e9184c13b7c2607fe6c30ae19403a7aff
 init version: de40ad0 (expected: fec3683)
 Security Options:
  seccomp
   Profile: default
 Kernel Version: 4.14.193-149.317.amzn2.x86_64
 Operating System: Amazon Linux 2
 OSType: linux
 Architecture: x86_64
 CPUs: 2
 Total Memory: 3.794GiB
 Name: ip-10-0-1-170.ap-northeast-2.compute.internal
 ID: SPCF:Y75R:B22D:BLRW:ELZ5:3O4B:HPDJ:R3JI:DSQM:W74R:TRA2:BXWE
 Docker Root Dir: /var/lib/docker
 Debug Mode: false
 Registry: https://index.docker.io/v1/
 Labels:
 Experimental: false
 Insecure Registries:
  127.0.0.0/8
 Live Restore Enabled: false

아래와 같은 docker run 명령어를 실행해서 제플린 개발환경을 구성한다.

`docker run -p 4040:4040 -p 8080:8080 --privileged=true -v $PWD/logs:/logs -v $PWD/notebook:/notebook -e ZEPPELIN_NOTEBOOK_DIR='/notebook' -e ZEPPELIN_LOG_DIR='/logs' -d apache/zeppelin:0.8.1 /zeppelin/bin/zeppelin.sh`

In [ ]:
[ec2-user@ip-10-0-1-170 ~]$ docker run -p 4040:4040 -p 8080:8080 --privileged=true -v $PWD/logs:/logs -v $PWD/notebook:/notebook \
> -e ZEPPELIN_NOTEBOOK_DIR='/notebook' \
> -e ZEPPELIN_LOG_DIR='/logs' \
> -d apache/zeppelin:0.8.1 \
> /zeppelin/bin/zeppelin.sh
Unable to find image 'apache/zeppelin:0.8.1' locally
0.8.1: Pulling from apache/zeppelin
7b722c1070cd: Pull complete
5fbf74db61f1: Pull complete
ed41cb72e5c9: Pull complete
7ea47a67709e: Pull complete
7ba34fd9f5e0: Pull complete
8f2f09b83582: Pull complete
40260f0a8f69: Pull complete
48946af5572c: Pull complete
8b38acee7e8d: Pull complete
a806f41d7e41: Pull complete
7dcaf396dead: Pull complete
8db355f40e66: Pull complete
Digest: sha256:a3a90ec1579f5171ebac565e739547b885ed75efc1ec7581128ec1033a4496cb
Status: Downloaded newer image for apache/zeppelin:0.8.1
9276b4344e403414c354ec41816abcd6005e637836d31f691319323b672fe279

[ec2-user@ip-10-0-1-170 ~]$ docker ps
CONTAINER ID        IMAGE                   COMMAND                  CREATED             STATUS              PORTS                                            NAMES
9276b4344e40        apache/zeppelin:0.8.1   "/usr/bin/tini -- /z…"   7 seconds ago       Up 5 seconds        0.0.0.0:4040->4040/tcp, 0.0.0.0:8080->8080/tcp   confident_swirles

위와 같이 정상적으로 도커를 실행하고 나면 웹브라우저를 열고 `[ec2 public ip]:8080` 으로 접속해서 제플린으로 정상 접근가능한지 확인해본다.

접속한 다음에 `Create new note`을 클릭해서 spark 노트북을 새로 만들어준다. 그런 다음에 아래 그림과 같이 첫번째 블럭에 spark 문자열을 입력후 실행해본다. 실행하면 sparksession이 구동되는 것을 확인할 수 있다.

![1](https://user-images.githubusercontent.com/41605276/101123798-f3b92e00-3638-11eb-8be0-71e23ed84eae.PNG)

그런 다음에 웹브라우저를 하나 새로열고 `[ec2 public ip]:4040`를 입력해서 아래 그림과 같이 spark ui에 정상접속하는지 확인해본다.


![2](https://user-images.githubusercontent.com/41605276/101123952-4e528a00-3639-11eb-9a9e-889d83083460.PNG)

생성한 제플린 노트북에서 아래와 같은 코드로 실습을 해본다.

'티아카데미 아파치 스파크 입문과 활용 - 데이터 처리 실습 제플린 파일' 폴더에서 'test.json' 참고할것

In [ ]:
%spark
//sparksession driver process,어플리케이션 기준 
spark


%spark
//한개의 컬럼과 1000개의 row(0~999)를 생성한 데이터 프레임을 만든다
val myRange = spark.range(1000).toDF("number")


%spark
//transformation - narrow dependency
val divisBy2 = myRange.where("number % 2 = 0")

%spark
//action
divisBy2.count()


%sh
//데이터 다운로드
wget https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/flight-data/csv/2015-summary.csv


%sh
//데이터 확인
head /zeppelin/2015-summary.csv


%spark
//CSV데이터 데이터 프레임으로 바로 읽기, visit to http://localhost:4040
val flightData2015 = spark.read.option("inferSchema", "true").option("header", "true").csv("/zeppelin/2015-summary.csv")


%spark
//3건만 가져오기
//take같이 action이 발생할때 spark 대시보드에 로그가 남는다.
flightData2015.take(3)


%spark
//실행계획 확인하기
flightData2015.sort("count").explain()


%spark
//count필드로 정렬해서 2개 가져오기
flightData2015.sort("count").take(2)


%spark
//SQL을 사용하기 위해 임시 view만들기
// tempview나 temptable은 sparkcontext안에서만 유효하기 때문에 sparkcontext가 종료되면 이것들도 같이 날아간다.
flightData2015.createOrReplaceTempView("flight_data_2015")


%spark
//SQL로DEST_COUNTRY_NAME 기준으로 몇건인지 확인하는 spark sql실행하기
val sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")


%spark
//데이터 프레임 통계 보기
sqlWay.describe().show()



%spark
//데이터 프레임으로 실행계획 확인하기
val dataFrameWay = flightData2015
  .groupBy('DEST_COUNTRY_NAME)
  .count.explain


%spark
//sql과 dataframe실행계획 비교하기           
sqlWay.explain


%spark
//sql질의로 max값 가져와서 take로 한건만 확인하기           
spark.sql("SELECT max(count) from flight_data_2015").take(1)


%spark
//데이터 프레임에서 max함수 이용하여 count의 최대값 가져오기           
import org.apache.spark.sql.functions.max

flightData2015.select(max("count")).take(1)


%spark
//spark sql로 DEST_COUNTRY_NAME 을 집계연산을 수행하여 합계를 구하여 큰 순서대로 5건 확인하기           
val maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")



%spark
//sql결과 확인하기           
maxSql.explain
maxSql.show()



%spark
//동일한 쿼리를 dataframe으로 구현해 봅니다.  End-to-End example
flightData2015
.groupBy("DEST_COUNTRY_NAME")
.sum("count")
.withColumnRenamed("sum(count)", "destination_total")
.sort(desc("destination_total"))
.limit(5)
.explain


%spark
flightData2015
.groupBy("DEST_COUNTRY_NAME")
.sum("count")
.withColumnRenamed("sum(count)", "destination_total")
.sort(desc("destination_total"))
.limit(5)
.show()


%spark
//임의의 값을 가지고 데이터 프레임 생성하기
val data = Seq(("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"),
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"),
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"),
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico"))

import spark.sqlContext.implicits._
val df = data.toDF("Product","Amount","Country")
df.show()

%spark
//각 제품의 각 국가로 수출 된 총 금액을 얻으려면 제품 별 그룹화, 국가 별 피봇 팅 및 금액 합계를 얻어온다
val pivotDF = df.groupBy("Product").pivot("Country").sum("Amount")
pivotDF.show()

%spark
//spark2.0에서 성능향상을 위해 pivot대상열을 선언함 
val countries = Seq("USA","China","Canada","Mexico")
val pivotDF = df.groupBy("Product").pivot("Country", countries).sum("Amount")
pivotDF.show()

%spark
//성능 향상을 위해 두단계 집계를 사용함 
val pivotDF = df.groupBy("Product","Country")
      .sum("Amount")
      .groupBy("Product")
      .pivot("Country")
      .sum("sum(Amount)")
pivotDF.show()
           
%spark
//stack기능을 이용하여 주요 국가에 대해 unpivot을 수행
val unPivotDF = pivotDF.select($"Product",
expr("stack(4, 'Canada', Canada, 'China', China, 'Mexico', Mexico, 'USA', USA) as (Country,Total)"))
.where("Total is not null")
unPivotDF.show()
           
  
//%sql로 하면 sparksql 커널로 잡히게 된다.
%sql
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5

- 참고사항 : sparksession과 sparkcontext

1) 둘의 차이점 결론

spark.sparkContext = RDD 생성시 사용하는 API 객체로 application 전체의 실행 관련 정보 집약 객체로 스케줄러 등이 포함된 개념이다.

spark.sparkSession = DataFrame 생성시 사용하는 API 객체로 sparkcontext에 세션정보가 추가로 포함된 개념이다.

RDD를 생성하려면 SparkContext 객체를 생성하고, DataFrame 또는 DataSet을 생성하려면 SparkSession 객체를 생성해야 한다. 그런데 sparksession 객체 안에는 sparkcontext 객체가 포함돼 있기 때문에 RDD를 만들때나 데이터프레임을 만들때나 상관없이 sparksession 객체를 생성하면 된다.

2) SparkSession

Spark2.0 이후부터는 SparkSession을 사용해서 Spark Dataset, DataFrame API를 사용할 수 있다. 또한 SparkContext에서 사용 가능한 모든 기능은 SparkSession에서도 사용이 가능하다.


3) sparkcontext

SparkSession를 이용해서 SparkContext()를 사용하고 싶다면 SparkSession에서 sparkContext() 메서드를 사용하면 SparkContext를 사용할 수 있다.SparkConext는 Spark 클러스터에 대한 연결을 나타내며 해당 클러스터에서 RDD, broadcast, accumulator 등의 변수를 사용하여 사용이 가능하다. JVM 당 하나의 SparkContext만 활성화 할 수 있다.

- RDD란

1) RDDs (Resilient Distributed Datasets) is Data Containers


2) All the different processing components in Spark share the same abstraction called RDD


spark의 모든 프로세싱은 RDD 기반으로 모든 데이터를 share한다고 이해하면 된다.


3) As applications share the RDD abstraction, you can mix different kind of transformations to create new RDDs


4) Created by parallelizing a collection or reading a file


5) Fault tolerant


- RDD 개념이해를 위한 예시 : log mining

HDFS에 로그가 쌓여있다고 가정하자. 데이터를 line by line으로 읽는데 'ERROR'가 포함된 문자열을 detect하고 싶은것이다.

'ERROR'가 포함된 RDD를 따로 만들고, 거기에서 map작업을 하는데 '\t' split했을때 두번째 문자열을 message 변수로 가져오고 그거를 캐싱을 해서 거기에서 foo가 몇개이고 bar가 몇개인지 filter해서 count한다.

RDD가 변환시킬때마다 transformation이 일어난다. 변환될때마다 transformed RDD가 계속 생성된다. 그리고 그거를 count를 해서 action을 하게되면 driver로 이벤트가 날아가게 된다.

![3](https://user-images.githubusercontent.com/41605276/101245378-cd39e680-374f-11eb-8f5e-077fb7ceef86.png)

위에 그림에서 driver는 자바에서 메인함수를 가지는 어플리케이션이라고 할 수 있다. driver에 code가 submit이 되면 클러스터안에 워커한테 task들이 분배가 될것이다. 그리고 cache를 위에서 사용했는데 worker의 메모리 영역에서 일부영역을 캐시영역으로 활용하게 된다. 그리고 spark의 task들이 완료되면 결과값을 driver로 던져주게 된다.


- RDD fault tolerance가 뭐냐

HDFS는 매번 읽고 쓰고를 하는데 읽고 쓰고 하는 만약에 iteration한 job이 있는데 이게 중간에 갑자기 문제가 생겨서 다시 이 job을 재개한다면 mapreduce는 다시 디스크를 읽어서 중간산출물을 다시 읽게 된다. 그런다음에 다시 computing을 하게 된다. 그러면 spark 같이 인메모리 구조에서는 이런 recomputing을 어떻게 하냐. RDD는 리니지 트레킹이라는 개념이 있다. 아래 그림과 같이 데이터가 변형이 가해질때마다 RDD가 새로 생성이 된다. 그래서 연산중간에 문제가 생기면 바로이전 단계 RDD를 트레킹해서 리트라이 할 수 있도록 한다.

"RDDs track the transformations used to build them (their lineage) to recompute lost data"

![4](https://user-images.githubusercontent.com/41605276/101246141-c2358500-3754-11eb-93ba-1c2601812b08.PNG)


- scala vs java

spark은 jvm 기반이기 때문에 자바 스칼라 모두 가능하고, 스칼라 언어의 간결성과 함수형 프로그래밍의 장점을 살리고자 주로 스칼라를 spark에서는 사용한다. spark-submit하고 spark job의 프로세싱은 jvm 위에서 모두 실행이 된다.

![5](https://user-images.githubusercontent.com/41605276/101246292-beeec900-3755-11eb-89ac-9f12896b4311.PNG)


- 참고로 spark 데이터프레임만 사용한다면 java, scala, python 언어의 성능차이는 최근에는 거의 없다고 할 수 있다.


- scala cheat sheet

In [ ]:
//variables
var x: Int = 7
    
var x = 7 
// type inferred

val y = "hi"
// read-only

// Functions
def square(x: Int): Int = x*x
def square(x: Int): Int = {
    x*x // last line returned
}
    
// Collections and closures
val nums = Array(1, 2, 3)
nums.map((x: Int) => x + 2) 
// => Array(3, 4,5)

nums.map(x => x + 2) 
// => same

nums.map(_ + 2) 
// => same

nums.reduce((x, y) => x + y) 
// => 6

nums.reduce(_ + _) 
// => 6

- RDD operations

spark은 최적의 DAG를 action 시점에서 찾기 위해서 lazy operation을 한다.      ** DAG : 특정 job이 돌기위해서 실행되는 프로세스

사용자가 이것저것 transformation을 하더라도 실제로 action단계에서 transformation의 실행계획이 최적화 되는것이다.

1) Transformations (e.g. map, filter, groupBy,join)

Lazy operations to build RDDs from other RDDs

2) Actions (e.g. count, collect, save)

Return a result or write it to storage

![6](https://user-images.githubusercontent.com/41605276/101247531-7affc200-375d-11eb-84d8-3db3cdcfd382.PNG)

action이 수행되고 결과값은 driver로 return하거나 특정 스토리지에 저장할 수 있다.


- RDD와 Dataframe

RDDs provide a low level interface into Spark

DataFrames have a schema

DataFrames are cached and optimized by Spark

DataFrames are built on top of the RDDs and the core Spark API

![7](https://user-images.githubusercontent.com/41605276/101247772-b484fd00-375e-11eb-814d-a6d29aa1c1fe.PNG)

- spark Dataframe 샘플코드

코드가 상당히 직관적이고 쉽다.

In [ ]:
// Create a new DataFrame that contains “students”
students = users.filter(users.age < 21)

//Alternatively, using Pandas-like syntax
students = users[users.age < 21]

//Count the number of students users by gender
students.groupBy("gender").count()

// Join young students with another DataFrame called logs
students.join(logs, logs.userId == users.userId,"left_outer")

RDD와 Dataframe 샘플코드 비교

RDD같은 경우에는 \t으로 split을 한다음에 몇번째 array의 무슨값인지 다 지정을 해줘야 하는데 Dataframe은 스키마를 갖고 있기 때문에 Dataframe은 그런식으로 지정을 해줄 필요가 없다.

1) RDD

In [ ]:
data = sc.textFile(...).split("\t")
data.map(lambda x: (x[0], [int(x[1]), 1]))
.reduceByKey(lambda x, y: [x[0] + y[0], x[1] + y[1]])
.map(lambda x: [x[0], x[1][0] / x[1][1]])
.collect()

2) DataFrame

In [ ]:
sqlCtx.table("people").groupBy("name").agg("name",avg("age")).collect()